# IAP 2024: Advanced Optimization Session

This notebook is built based on the material developed by [Shuvomoy Das Gupta](https://github.com/Shuvomoy/MIT_IAP_2022_15_S60_Advanced_Optimization_Session_Shuvo)

In [ ]:
# using Pkg;
# Pkg.add.(["Convex", "JuMP", "Images", "DelimitedFiles"]);

# Session 1

We will cover the following topic today.

# Topics

---
$\textsf{Session 1: Convex Optimization and Extension}$
---

* How to model an optimization problem in a tractable manner


* Tractable optimization models


* Convex optimization through `Convex.jl` and `JuMP`


* Modeling examples of common convex programs that shows up in OR


* What to do when the problem is nonconvex


---
$\textsf{Session 2: Discrete Optimization: Basic and Advanced}$
---

* Mixed Integer Linear Programs (MILP): basic


* How MILPs are solved: Branch-and-Bound algorithm


* How do we exploit structures to speed up MILPs


* Solving Mixed Integer Quadratic Problems

## Modeling an optimization problem 

* We have our research problem, that usually come from a practical problem rooted in reality. 

* While modeling the problem, we want to stay as close to reality as possible, but without making the model intractable. 

* This is a balance. Safety-critical constraints have to be modeled exactly, where soft constraints can be relaxed.

* It is a good practice to be considerate during the modeling phase so we end up with a model that is practically tractable.

### Some good resources on how to model optimization problems

* Model building in mathematical programming by H Williams ([Link](https://www.amazon.com/Model-Building-Mathematical-Programming-Williams/dp/1118443330))
  * A very practical book for modeling LP and MILP
  

* Optimization models by G Calafiore and L El Ghaoui ([Link](https://www.amazon.com/Optimization-Models-Giuseppe-C-Calafiore/dp/1107050871))
  * Good book for modeling convex optimization problems, especially Chapter 9, 10, 11  
  
  
  
* Applications of optimization with Xpress-MP by C Guéret, C Prins, and M Sevaux ([Link](https://www2.isye.gatech.edu/~sahmed/isye3133b/XpressBook.pdf))
  * Excellent for modeling integer optimization models, especially Chapter 3, with lot of shortcuts and hacks  

## Modeling an optimization problems

At present the following types of problems are *practically tractable*. Whenever we are modeling an optmization problem, we should try to ensure that our model is one of the following:

1. Linear programs (LP)



2. Quadratic convex programs (QCP)



3. Second order cone programs (SOCP)



4. Semidefinite programs (SDP)



5. Mixed-integer linear programs (MILP)

## My modeling strategies

* **Solve the smaller optimization problem first**

> Thomson's Rule for First-Time Telescope Makers: "It is faster to *make a four-inch mirror then a six-inch mirror* than to *make a six-inch mirror*."

I follow a modified version of Thomson's rule

It is faster to **solve a smaller simpler optimization model and then the larger complicated optimization model** than to *solve a larger complicated optimization model*.

* **Do not optimize prematurely**

> "The real problem is that programmers have spent far too much time worrying about efficiency in the wrong places and at the wrong times; premature optimization is the root of all evil (or at least most of it) in programming.” 
> - Donald Knuth, The Art of Computer Programming

## Milestones in optimization


* 1947: G. Dantzig, who works for US air-forces, presents the Simplex method for solving LP-problems



* 1948: J. Von Neumann establishes the theory of duality for LP-problems



* 1951: H.W. Kuhn and A.W. Tucker reinvent Karush's optimality conditions (known as KKT conditions) 



* 1951: H. Markowitz presents his portfolio optimization theory => (1990 Nobel prize)



* 1954: L.R. Ford's and D.R. Fulkerson's research on network problems => start of combinatorial optimization



* 1960-1970: Many of the early works on first-order optimization algorithms are done (mostly developed in Soviet Union)



* 1983: Nesterov comes up with accelerated gradient descent



* 1984: N. Karmarkar's polynomial time algorithm for LP-problems begins a boom period for interior point methods



* 1990s: Semidefinite optimization theory


* 2010-present: First-order methods become very hot again due to machine learning


* 2014: Performance estimation problem: computer-assisted design and analysis of optimization algorithms

## General convex optimization problem

Most authoritative reference is Convex Optimization by Boyd and Vandenberghe ([pdf](https://web.stanford.edu/~boyd/cvxbook/)).

Convex optimization problems are defined through the notion of convex set.

**Convex set**

<img src="figures/convex/convex_set.png" alt="Drawing1" style="width: 700px;"/>

**Convex function**

A function  is convex if and only if the region above its graph is a convex set.


<img src="figures/convex/convex_function.png" alt="Drawing1" style="width: 500px;"/>

## Standard form of a convex optimization problem

A general convex optimization problem has
the following form:
$$
\begin{array}{ll}
\underset{x\in\mathbf{R}^{d}}{\mbox{minimize}} & f_{0}(x)\\
\mbox{subject to} & a_{i}^{\top}x=b_{i},\quad i=1,\ldots,p,\\
 & f_{i}(x)\leq0,\quad i=1,\ldots,m.
\end{array}
$$
 where the equality constraints are linear and the functions $f_{0},f_{1},\ldots,f_{m}$
are convex. 

## Convex optimization

* a subclass of optimization problems that includes LP as special case


* convex problems can look very difficult (nonlinear, even
nondifferentiable), but like LP can be solved very efficiently


* convex problems come up more often than was once thought


* many applications recently discovered in control, combinatorial
optimization, signal processing, communications, circuit design,
machine learning, statistics, finance, . . .

## General approaches to using convex optimization

* Pretend/assume/hope $f_i$ are convex (around minimum) and proceed $\Rightarrow \textsf{Machine Learning in Practice}$ 
    - easy on user (problem specifier)
    - but lose many benefits of convex optimization
    - risky to use in saftey-critical application domain
    - an important example `Adam`: one of the most heavily used solver for deep learning.  (also see [Link](https://arxiv.org/pdf/1804.10587.pdf))
    
    
* Verify problem is convex (around minimum) before attempting solution $\Rightarrow \textsf{Machine Learning Theory}$ 
     - but verification for general problem description is hard  
     
     
* Model the problem in a way that results in a convex formulation  $\Rightarrow \textsf{Operations Research}$ 
    - user needs to follow a restricted set of rules and methods
    - convexity verification is automatic

Each has its advantages, but we focus on 3rd approach.

## How can you tell if a problem is convex?

* Need to check convexity of a function

Approaches:
* use basic definition, first or second order conditions, e.g., $\nabla^2 f(x) \succeq 0$
* via convex calculus: construct $f$ using
  - library of basic examples or atoms that are convex
  - calculus rules or transformations that preserve convexity

## Basic convex functions (convex atoms)

* $x^{p}$ for $p\geq1$ or $p\leq0$; $-x^{p}$ for $0\leq p\leq1$

* $e^{x},$$-\log x$, $x\log x$

* $a^{\top}x+b$

* $x^{\top}x$; $x^{\top}x/y$ (for $y>0$); $\sqrt{x^{\top}x}$

* $\|x\|$ (any norm)

* $\max(x_{1},\ldots,x_{n})$

* $\log(e^{x_{1}}+\ldots+e^{x_{n}})$

* $\log\det X^{-1}$ (for $X\succ0$)

These are also called *atom*s because they are building block of much more complex convex functions. There are many such atoms, most convex programs in practice can be built from these atoms. A more complete list can be found [here](https://jump.dev/Convex.jl/stable/operations/). 

When modeling a problem, we aim to construct a model in such a way so that the building blocks of the functions are the atoms above. 

## Convex calculus rules

* nonnegative scaling: if $f$ is convex then $\alpha f$ is convex
if $\alpha\geq0$



* sum: if $f$ and $g$ are convex, then so is $f+g$



* affine composition: if $f$ is convex, then so is $f(Ax+b)$



* pointwise maximum: if $f_{1},f_{2},\ldots,f_{m}$ are convex, then
so is $f(x)=\max_{i\in\{1,\ldots,m\}}f_{i}(x)$ 

* pointwise supremum: if $f(x,y)$ is convex in $x$ for all $y \in S$, then $g(x) = \textrm{sup}_{y \in S}{f(x,y)}$ is convex



* partial minimization: if $f(x,y)$ is convex in $(x,y)$ and $C$
is convex, then $g(x)=\min_{y\in C}f(x,y)$ is convex



* composition: if $h$ is convex and increasing and $f$ is convex,
then $g(x)=h(f(x))$ is convex

## The function that may show up in your model can be much more complicated

Many complicated convex functions that we can use in the modeling phase can build from the atoms through convex calculus.

* piecewise-linear function: $f(x)=\max_{i=1,\ldots,k}(a_{i}^{\top}x+b_{i})$



* $\ell_{1}$-regularized least-squares cost: $\|Ax-b\|_{2}^{2}+\lambda\|x\|_{1}$
with $\lambda\geq0$


* support-function of a set: $S_C(x)= \max_{y \in C}{x^\top y}$ where $C$ is any set



* distance to convex set: $f(x)=\min_{y\in C}\|x-y\|_{2}$

## Basic modeling steps

* When modeling a problem we face many choices how to mathematically model a certain reality


* During the modeling phase try to select your functions with their building blocks as convex atoms


* When modeling some operation on a mathematical object try to use convex calculus rules


* If you can follow the last two steps, you will have a convex problem


* If you have a nonconvex problem (that is not mixed integer linear program), you can try one of the two things:
   - Work with tightest convex approximation of the nonconvex problem (will see one example)
   - *Sequential convex programming*, a local optimization method for nonconvex problems that leverages convex optimization (works suprisingly well in my experience)

## Linear programs

$$
\begin{array}{ll}
\underset{x\in\mathbf{R}^{d}}{\mbox{minimize}} & c^{\top}x\\
\mbox{subject to} & Ax=b,\\
 & Cx\geq d.
\end{array}
$$

Shows up when we are modeling system that can be completely described by linear equalities and inequalities.

## Quadratic convex programs (QCP)

$$
\begin{array}{ll}
\underset{x\in\mathbf{E}}{\mbox{minimize}} & c^{\top}x+x^{\top}\underbrace{Q_{0}}_{\succeq0}x\\
\mbox{subject to} & Ax=b,\\
 & x^{\top}\underbrace{Q_{i}}_{\succeq0}x+q_{i}^{\top}x+r_{i}\leq0,\quad i=1,\ldots,p.
\end{array}
$$

#### Where does it show up?

QCP mostly show up in data fitting problems such as least-squares and its variants.

## Second order cone programs (SOCP)

$$
\begin{array}{ll}
\underset{x\in\mathbf{R}^{d}}{\mbox{minimize}} & c^{\top}x\\
\mbox{subject to} & \|A_{i}x+b_{i}\|_{2}\leq q_{i}^{\top}x+d_{i},\quad i=1,\ldots,p,\\
 & Cx\leq f.
\end{array}
$$

* SOCP is a generalization of LP and QCP
*  Allows for linear combinations of variables to be constrained inside a special convex set, called a second-order cone

<img src="figures/convex/socp.png" alt="image-20220117142714899" style="zoom: 50%;" />

* Shows up in: geometry problems, approximation problems, chance-constrained linear optimization problems

## Semidefinite programs (SDP)

Recall that LP in standard form is:
$$
\begin{array}{ll}
\underset{x\in\mathbf{R}^{d}}{\mbox{minimize}} & c^{\top}x\\
\mbox{subject to} & a_{i}^{\top}x=b_{i},\quad i=1,\ldots,m,\\
 & x\succeq0.
\end{array}
$$
An SDP is a generalization of LPs. Standard form of an SDP is:
$$
\begin{array}{ll}
\underset{X\in\mathbf{R}^{d\times d}}{\mbox{minimize}} & \mathbf{tr}(CX)\\
\mbox{subject to} & \mathbf{tr}(A_{i}X)=b_{i},\quad i=1,\ldots,m,\\
 & X\succeq0.
\end{array}
$$

Here $X, C, A_{i}$ are symmetric matrices.

SDP shows up in:
* sophisticated relaxations (approximations) of non-convex problems,
* in control design for linear dynamical systems,
*  in system identification,
* in algebraic geometry, and
* in matrix completion problems.

## Mixed-integer linear programs (MILP) (Session 2)

$$
\begin{array}{ll}
\underset{x,y}{\mbox{minimize}} & c^{\top}x+d^{\top}y\\
\mbox{subject to} & Ax+By=b,\\
 & Cx+Dy\geq f,\\
 & x\succeq0,\\
 & x\in\mathbf{R}^{d},\\
 & y\in \mathbf{Z}^{n}.
\end{array}
$$

* Nonconvex, but "practically tractable" in many situations
* MILP solvers have gained a speed-up factor of 2 trillion in the last 40 years!
* Wide modeling capability
* Usually shows up in situations where
  * modeling do/don't do decisions
  * logical conditions
  * implications (if something happens do this)
  * either/or

# Modeling and solving common convex models in `Julia`

### Example of QCP: portfolio optimization


Assume that we have a portfolio with $n$ assets at the beginning of time period $t$.

Given some forecasts on risks and expected returns we try to find the optimal portfolio that rebalances the portfolio to achieve a good balance between expected risk (variance) $x^\top \Sigma x$ and returns $\mu^\top x$​​.

In it's simplest form we want to solve:
$$
\begin{array}{ll} \text{minimize} &  	 \gamma (x^\top \Sigma x) -\mu^\top x \\
\text{subject to} &  1^\top x = d + 1^\top x^0 \\
                    &  x  \geq 0,
 \end{array}
$$
with variable $x \in \mathbf{R}^n$, $\mu$ forecasted (expected) returns, $\gamma > 0 $ risk aversion parameter.

* $x^0_i$ represents the initial investment in asset $i$ and $d$ represents the cash reserve.
* The equality constraint tells us that the sum of the new allocation vector $x$​ has to equal the initial allocation plus the cash reserve
* the covariance matrix of our risk model is given by $\Sigma \in \mathbf{S}_+^n$​.

In [ ]:
# Load the packages
using MosekTools, Mosek, Random, Convex, JuMP, Images, DelimitedFiles 
# If you do not have Mosek then instead run
# using COSMO, Random, Convex, JuMP, Images, DelimitedFiles 

In [ ]:
# Load the data
include("data//convex//portfolio_data.jl");

### Solving the problem using `Convex.jl`

To solve this problem, we will use `Convex.jl`, which is a specialized `Julia` package for solving convex optimization problem.

Declare the variable $x$.

In [ ]:
x = Variable(n)

Let us now the define the objective $ \gamma (x^\top \Sigma x) -\mu^\top x $.

In [ ]:
objective = γ*quadform(x,Σ) - dot(x,μ)

Note, the list of all the functions `Convex.jl` can model directly is available at [https://jump.dev/Convex.jl/stable/operations/#Linear-Program-Representable-Functions](https://jump.dev/Convex.jl/stable/operations/#Linear-Program-Representable-Functions).

Add the constraint $1^\top x = d + 1^\top x^0$.

In [ ]:
constraint_linear = ( sum(x) == d + sum(x0) )

Now add the positivity constraint.

In [ ]:
constraint_pos = ( x >= 0 )

Next, combine everything in a `problem`.

In [ ]:
problem = minimize(objective, [constraint_linear, constraint_pos])

Now time to solve the problem!

In [ ]:
solve!(problem, Mosek.Optimizer)

Get the solution now.

In [ ]:
@show x_sol = Convex.evaluate(x);

Get the optimal value.

In [ ]:
@show p_star = Convex.evaluate(objective)

So, combining everything together, the code to solve the portfolio optimization problem using `Convex.jl` would be:

In [ ]:
## Define the problem
x = Variable(n);
objective = γ*quadform(x,Σ) - dot(x,μ);
constraint_linear = ( sum(x) == d + sum(x0) );
constraint_pos = ( x >= 0 );
problem = minimize(objective, [constraint_linear, constraint_pos]);

## Solve the problem
solve!(problem, Mosek.Optimizer);

## Extract the solution
x_sol = Convex.evaluate(x);
p_star = Convex.evaluate(objective);

### Solving the problem using `JuMP`

There is nothing special solving a problem via `Convex.jl` over `JuMP`. We could solve the same problem using `JuMP` with the following code.

In [ ]:
## Define the problem
using JuMP
model = Model(Mosek.Optimizer)
@variable(model, xs[1:n] >= 0)
@objective(model, Min,  γ * xs'*Σ*xs  - μ' * xs);
@constraint(model, sum(xs) .== d + sum(x0))

## Solve the problem
optimize!(model)

## Get the optimal solution
x_sol_JuMP = value.(xs)
p_star_JuMP = objective_value(model)

Because we used the same solver, we hope to get the same optimal solution.

In [ ]:
norm(x_sol - x_sol_JuMP)

### When to use `JuMP` vs `Convex.jl`?

* Usually `JuMP`'s model creation time is somewhat faster than `Convex.jl`, so if you are working on a problem, where performance is key, then using `JuMP` is a good idea.
* If you are in the prototyping phase of your model and you feel that you are working with a lot of convex-ish functions, then working with `Convex.jl` might be a good idea.
  * For example, suppose, in the last problem you are interested in finding a sparse portfolio with the risk being less than some bound. Within convex optimization framework, one way doing this is by minimizing the $\ell_1$ norm of $x$, i.e., consider the modified objective  $-\mu^\top x + 0.1 \cdot \| x\|_1$ (to balance the $\mu^\top x$ and the sparsity) and the additional constraint $x^\top \Sigma x \leq \delta := 0.5$ (to ensure the expected risk no larger than $\delta$). If we want to do this modification via `JuMP`, then the right approach is reformulate the  $\ell_1$​ norm via epigraph etc, but `Convex.jl` will take this term as it is.


$\|x\|_1$ = `norm(x,1)`

$x^\top \Sigma x$ = `quadform(x_sparse,Σ)`

Exercise:

In [ ]:
## Fill in the 😕s
δ = 0.5
x_sparse = Variable(n)
objective_sparse = 😕
constraint_linear = ( sum(x_sparse) == d + sum(x0) )
constraint_pos = ( x_sparse >= 0 )
constraint_additional = ( 😕 <= δ)
problem_sparse = minimize(objective_sparse, [constraint_linear, constraint_pos, constraint_additional])

## Solve the problem
solve!(problem_sparse, Mosek.Optimizer)

## Extract the solution
x_sol_sparse = Convex.evaluate(x_sparse)
p_star_sparse = Convex.evaluate(objective_sparse)

Answer:

In [ ]:
δ = 0.5
x_sparse = Variable(n)
objective_sparse =   - dot(x_sparse, μ) + 0.1 * norm(x_sparse,1)
traint_linear = ( sum(x_sparse) == d + sum(x0) )
constraint_pos = ( x_sparse >= 0 )
constraint_additional = ( quadform(x_sparse,Σ) <= δ)
problem_sparse = minimize(objective_sparse, [constraint_linear, constraint_pos, constraint_additional])

## Solve the problem
solve!(problem_sparse, Mosek.Optimizer)

## Extract the solution
x_sol_sparse = Convex.evaluate(x_sparse)
p_star_sparse = Convex.evaluate(objective_sparse)

In [ ]:
using Plots
plot([1:n],[x_sol_JuMP x_sol_sparse], xlabel = "k", ylabel = "x[k]", label = [ "x original" "x sparse"])

### Example of SOCP: time series analysis

A time series is a sequence of data points, each associated with a time. In our example, we will work with a time series of daily temperatures in the city of Melbourne, Australia over a period of a few years. Let $\tau$ be the vector of the time series, and $\tau_i$ denote the temperature in Melbourne on day $i$. Here is a picture of the time series:

In [ ]:
using DelimitedFiles, Plots
τ = readdlm("data//convex//melbourne_temps.txt", ',')
n = size(τ, 1)
plot(1:n, τ[1:n], ylabel="Temperature (°C)", label="data", xlabel = "Time (days)", xticks=0:365:n)

A simple way to model this time series would be to find a smooth curve that approximates the yearly ups and downs.
We can represent this model as a vector $x$ where $x_i$ denotes the predicted temperature on the $i$-th day.
To force this trend to repeat yearly, we simply want to impose the constraint

$$
 x_i = x_{i + 365}
$$

for each applicable $i$.

We also want our model to have two more properties:

- The first is that the temperature on each day in our model should be relatively close to the actual temperature of that day and equal if possible.
- The second is that our model needs to be smooth, so the change in temperature from day to day should be relatively small. The following objective would capture both properties:

$$
\sum_{i=1}^{n}|x_{i}-\tau_{i}|+\lambda\sqrt{\sum_{i=2}^{n}(x_{i}-x_{i-1})^{2}}=\|x-\tau\|_{1}+\lambda\|Ax\|_{2},
$$
 where $A$ is a matrix of size $(n-1)\times n$ with $A_{i,i}=-1,A_{i,i+1}=1$
for $i=1,\ldots,n-1$ and the rest of the elements being zero. So,
the optimization problem we want to solve is: 

$$
\begin{array}{ll}
\underset{x\in\mathbf{R}^{d}}{\mbox{minimize}} & \|x-\tau\|_{1}+\lambda\|Ax\|_{2}\\
\mbox{subject to} & x_{i}=x_{i+365},\quad i=1,\ldots,n.
\end{array}
$$
This is an SOCP, because it can be written as: 
$$
\begin{array}{ll}
\underset{x,u,t}{\mbox{minimize}} & \sum_{i=1}^{n}u_{i}+\lambda t\\
\mbox{subject to} & x_{i}=x_{i+365},\quad i=1,\ldots,n, \\
& \|Ax\|_{2}\leq t,\\
 & \vert x_{i}-\tau_{i}\vert\leq u_{i},\quad i=1,\ldots,n.
\end{array}
$$

Here, $\lambda$ is the smoothing parameter. The larger $\lambda$ is, the smoother our model will be.


We will solve this problem using `Convex.jl`, because it would allow us to input the problem directly without manually converting into the SOCP format. 

In [ ]:
## Solution using Convex.jl
x = Variable(n)

In [ ]:
eq_constraints = [ x[i] == x[i - 365] for i in 365 + 1 : n ];

In [ ]:
λ = 1 # smoothing parameter
# Define the matrix A
A = zeros(n-1,n)
for i in 1:n-1
    A[i,i] = -1
    A[i,i+1] = 1
end

In [ ]:
smooth_objective = norm(x-τ,1) + λ*norm(A*x,2)

In [ ]:
smooth_problem = minimize(smooth_objective, eq_constraints);

In [ ]:
solve!(smooth_problem, Mosek.Optimizer)

Let's plot our smoothed time estimate vs the original data.

In [ ]:
# Plot smooth fit
plot(1:n, τ[1:n], label="data")
plot!(1:n, Convex.evaluate(x)[1:n], linewidth=2, label="smooth fit",  ylabel="Temperature (°C)", xticks=0:365:n, xlabel="Time (days)")

## Matrix completion problem: how to reconstruct a distorted image

Suppose we are given a noisy image, and we want to clean the image up. In countless movies and tv shows, we have seen our hero figuring out who the killer is by polishing a very noisy image in seconds. In reality, this takes a while, and usually is done by solving an SDP. 

Let's take a look at an example. Suppose we are given this very noisy image. This image is the noisy version of a test image widely used in the field of image processing since 1973. See the wikipedia entry [here](https://en.wikipedia.org/wiki/Lenna) for more information.

In [ ]:
using Images
lenna = load("figures//convex//lena128missing.png")

A noisy image is basically a matrix, where lot of the pixels are not reliable. In a way, these unreliable pixels can be treated as missing entries of a matrix, because any $n\times n$ image is nothing but a $n \times n$ matrix. 

First, we convert the $128\times128$ image into a matrix. Here, through some precalculation, I have already filled in the missing entries with zero (for the sake of illustration).

In [ ]:
# convert to real matrices
Y = Float64.(lenna)

#### Goal 
Our goal is to find a the missing entries of this matrix $Y$ and thus reconstruct the image and hopefully figure out who this person is. 

#### Constraint to impose
* Ofcourse, we want to ensure that in the reconstructed image, call it $X$, for the available pixels, both images agree, i.e., for any $(i,j)$ in the index set of observed entries, we have $X_{i,j} = Y_{i,j}$. 

#### But how to fill in the rest of the entries? 

One reasonable of way of doing it finding the simplest image that fits the observed entries. Simplicity of an image when traslated to a matrix can correspond to a matrix with low rank. In other words, we want to minimize the rank of the decision matrix $X$, but subject to $X_{i,j} = Y_{i,j}$ for the observed pixels.

\begin{array}{ll}
\underset{X}{\mbox{minimize}} & \text{rank}(X)\\
\mbox{subject to} & X_{i,j}=Y_{i,j},\quad(i,j)\in\text{observed pixels of }Y
\end{array}

But this is a very hard problem and solving to certifiable global optimality is an active research area. As of now, problems beyond matrix size of $50\times 50$ cannot be solved in a tractable fashion. You can see Ryan Cory-Wright's paper on how to solve low-rank problems to certifiable global optimality [here](http://www.optimization-online.org/DB_FILE/2020/09/8031.pdf).

#### The best convex approximation of the problem

To work around this issue, rather than minimizing $\textrm{rank}(X)$ we minimize the best convex approximation of the rank function, which is the nuclear norm of $X$, denoted by $\| X \|_\star$, which is equal to the sum of singular values of the matrix $X$. So, we solve:

\begin{array}{ll}
\underset{X}{\mbox{minimize}} & \| X \|_\star \\
\mbox{subject to} & X_{i,j}=Y_{i,j},\quad(i,j)\in\text{observed pixels of }Y
\end{array}

The problem above can be formulated as an SDP.

Find the index set of the observed entries.

In [ ]:
observed_entries_Y = findall(x->x!=0.0, Y)

Find size of the matrix.

In [ ]:
N, N = size(Y)

Declare the variable $X$.

In [ ]:
X = Variable(N,N)

Add the objective $\| X \|_\star$.

In [ ]:
obj_SDP = nuclearnorm(X)

Now add the constraints $X_{i,j} = Y_{i,j}$ for all $(i,j) \in \texttt{observed_entries_Y}$.

In [ ]:
constraints_SDP = Convex.Constraint[]

In [ ]:
for index_i_j in observed_entries_Y
    i = index_i_j[1]
    j = index_i_j[2]
    push!(constraints_SDP,  X[i,j] == Y[i,j])
end

Create the problem now!

In [ ]:
problem_SDP = minimize(obj_SDP, constraints_SDP)

In [ ]:
# solve!(problem_SDP, Mosek.Optimizer)

# [💀] If you want to run it on your laptop, it needs about 30GB RAM and 937 seconds. 
# So think AGAIN!
#
# Constraints            : 40769           
# Scalar variables       : 49153 

```
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 40769           
  Affine conic cons.     : 1 (32896 rows)
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 49153           
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.01            
Lin. dep.  - primal attempts        : 1                 successes              : 1               
Lin. dep.  - dual attempts          : 0                 successes              : 0               
Lin. dep.  - primal deps.           : 0                 dual deps.             : 0               
Presolve terminated. Time: 0.04    
GP based matrix reordering started.
GP based matrix reordering terminated.
Optimizer  - threads                : 10              
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 32896           
Optimizer  - Cones                  : 1               
Optimizer  - Scalar variables       : 24769             conic                  : 24769           
Optimizer  - Semi-definite variables: 1                 scalarized             : 32896           
Factor     - setup time             : 72.44           
Factor     - dense det. time        : 56.18             GP order time          : 0.01            
Factor     - nonzeros before factor : 5.41e+08          after factor           : 5.41e+08        
Factor     - dense dim.             : 0                 flops                  : 1.19e+13        
ITE PFEAS    DFEAS    GFEAS    PRSTATUS   POBJ              DOBJ              MU       TIME  
0   1.3e+00  1.0e+00  1.0e+00  0.00e+00   0.000000000e+00   0.000000000e+00   1.0e+00  72.52 
1   4.8e-01  3.6e-01  5.9e-01  -9.71e-01  1.415377135e+02   1.431598397e+02   3.6e-01  150.96
2   4.0e-01  3.1e-01  3.4e-01  -3.34e-01  1.116403593e+02   1.121811844e+02   3.1e-01  239.37
3   7.0e-02  5.3e-02  1.1e-02  2.56e-01   1.532518635e+02   1.531753559e+02   5.3e-02  325.83
4   5.9e-03  4.5e-03  5.5e-04  1.07e+00   1.479916370e+02   1.479971145e+02   4.5e-03  410.94
5   1.3e-03  1.0e-03  5.8e-05  1.00e+00   1.481214349e+02   1.481225967e+02   1.0e-03  496.76
6   2.8e-05  2.2e-05  5.0e-08  1.00e+00   1.479726003e+02   1.479725623e+02   2.2e-05  583.75
7   4.6e-06  3.5e-06  1.2e-08  1.00e+00   1.479718423e+02   1.479718468e+02   3.5e-06  671.28
8   4.9e-07  3.7e-07  4.2e-10  1.00e+00   1.479711248e+02   1.479711253e+02   3.7e-07  758.13
9   7.4e-08  4.1e-08  1.3e-11  1.00e+00   1.479710892e+02   1.479710892e+02   3.5e-08  846.89
10  6.6e-10  7.2e-10  8.2e-15  1.00e+00   1.479710824e+02   1.479710824e+02   3.1e-10  936.97
Optimizer terminated. Time: 937.14 
```

The optimal solution will be:

In [ ]:
using Serialization
X_sdp_sol = deserialize("data//convex//optimal_X.dat");

# If you are not cheating like me, you should run:
# X_sdp_sol = Convex.evaluate(X);

In [ ]:
[lenna colorview(Gray, X_sdp_sol)]

Let us compare with the test set (original image):

In [ ]:
lenna_original = load("figures//convex//Lenna_(test_image).png")
[lenna colorview(Gray, X_sdp_sol) lenna_original]

If we had formulated and solve the problem using `JuMP` (this will be the homework), the time taken by it would be around 20 s, whereas time taken by `Convex.jl` is 937 s. The main reason behind the time difference is that, to use `Convex.jl` we could feed it the problem formulation rather than converting into an SDP form, so internally `Convex.jl` converts the model into an SDP programmatically. The converted SDP has the follwoing size:

```
The SDP size in Convex.jl (constructed internally Convex.jl) 
#-----------------------------------------------------------
 Constraints            : 40769                      
 Scalar variables       : 49153
```

Whereas, for `JuMP` we converted the problem ourselves into an SDP to feed it into `JuMP`. The final formulation would be much tighter than what `Convex.jl` does automatically. Note that we invested some time here by researching into google scholar and finding the right paper. The SDP in `JuMP` has the following size:
 
```
The SDP size in JuMP 
#------------------------------
  Constraints            : 8129                       
  Scalar variables       : 16257  
```  

## Sequential convex programming

* Solving a nonconvex problem using a local convex optimization method



* Convex portions of a problem are handled "exactly" and efficiently



* Sequential convex programming is a heuristic, it can fail



* Success often depend on a good starting point



* We consider the nonconvex problem: 


$$
\begin{array}{ll}
\underset{x\in\mathbf{R}^{d}}{\mbox{minimize}} & f_{0}(x)\\
\mbox{subject to} & f_{i}(x)\leq0,\quad i=1,\ldots,m,\\
 & h_{i}(x)=0,\quad j=1,\ldots,p.
\end{array}
$$


where $f_{0}$ and $f_{i}$ are possibly nonconvex, $h_{i}$ are
possibly non-affine. 

## Basic idea of Sequential convex programming

* Maintain the current iterate $x^{(k)}$ and convex trust region $\mathcal{T}^{(k)}$

* Form convex approximation $f_{i}^{\textrm{cvx}}$ of $f_{i}$ over
$\mathcal{T}^{(k)}$

* Form affine approximation $h_{i}^{\textrm{afn}}$ of $h_{i}$ over
$\mathcal{T}^{(k)}$

* Then update the iterate $x^{(k+1)}$ is the optimal point found by
solving the convex problem 
$$
\begin{array}{ll}
\underset{x\in\mathbf{R}^{d}}{\mbox{minimize}} & f_{0}^{\textrm{cvx}}(x)\\
\mbox{subject to} & f_{i}^{\textrm{cvx}}(x)\leq0,\quad i=1,\ldots,m,\\
 & h_{i}^{\textrm{afn}}(x)=0,\quad j=1,\ldots,p,\\
 & x\in\mathcal{T}^{(k)},
\end{array}
$$
which is a convex approximation of the original nonconvex problem.

## How to compute the approximations

Trust region is computed using $\mathcal{T}^{(k)}=\{x\mid\|x-x^{(k)}\|\leq\rho\}$

* 
$$
h_{i}^{\textrm{afn}}=h_{i}(x^{(k)})+\nabla h_{i}(x^{(k)})^{\top}(x-x^{(k)})
$$

* 
$$f_{i}^{\textrm{cvx}}=f_{i}(x^{(k)})+\nabla f_{i}(x^{(k)})^{\top}(x-x^{(k)})+\frac{1}{2}(x-x^{(k)})^{\top}P(x-x^{(k)})$$

where $P=\left[\nabla^{2}f(x^{(k)})\right]_{+}$ which is the PSD
part of Hessian